# Extract customer data
1. Remove records with NULL customer_id
2. Remove exact duplicate records
3. Remove duplicate records based on created_timestamp
4. CAST the columns to the correct data type
5. Write transformed data to the Silver schema

## 1. Remove records with NULL customer_id

In [0]:
%sql
SELECT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL;

## 2. Remove exact duplicate records

In [0]:
%sql
-- show duplicated records
SELECT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL
ORDER BY customer_id;

In [0]:
%sql
-- remove exact keywords with DISTINCT keyword
SELECT DISTINCT *
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL
ORDER BY customer_id;

In [0]:
%sql
-- not recommended as you can mix up info from different timestamps created
SELECT 
  customer_id,
  MAX(created_timestamp),
  MAX(customer_name),
  MAX(date_of_birth),
  MAX(email),
  MAX(member_since),
  MAX(telephone)
FROM gizmobox.bronze.v_customers
WHERE customer_id IS NOT NULL
GROUP BY customer_id
ORDER BY customer_id;

In [0]:
%sql
-- remove exact keywords with DISTINCT keyword
-- create temp view from this to reuse later
CREATE OR REPLACE TEMPORARY VIEW v_customers_distinct
AS
  SELECT DISTINCT *
  FROM gizmobox.bronze.v_customers
  WHERE customer_id IS NOT NULL
  ORDER BY customer_id;

In [0]:
%sql
SELECT 
  customer_id, 
  MAX(created_timestamp) AS max_created_timestamp
FROM v_customers_distinct
GROUP BY customer_id;

## 3. Remove duplicate records based on created_timestamp

In [0]:
%sql
-- join the v_customer_distinct and the table with the max created timestamps to remove duplicate records based on created_timestamps
WITH cte_max AS(
  SELECT 
    customer_id, 
    MAX(created_timestamp) AS max_created_timestamp
  FROM v_customers_distinct
  GROUP BY customer_id
)
SELECT
  t1.*
FROM v_customers_distinct t1
JOIN cte_max t2
ON t1.customer_id = t2.customer_id AND t1.created_timestamp = t2.max_created_timestamp;


## 4. CAST the column values to correct data types

In [0]:
%sql
WITH cte_max AS(
  SELECT 
    customer_id, 
    MAX(created_timestamp) AS max_created_timestamp
  FROM v_customers_distinct
  GROUP BY customer_id
)
SELECT
  CAST(t1.created_timestamp AS timestamp) created_timestamp,
  t1.customer_id,
  t1.customer_name,
  CAST(t1.date_of_birth AS date) date_of_birth,
  t1.email,
  CAST(t1.member_since AS date) member_since,
  t1.telephone
FROM v_customers_distinct t1
JOIN cte_max t2
ON t1.customer_id = t2.customer_id AND t1.created_timestamp = t2.max_created_timestamp;

## 5. Write data to delta table

In [0]:
%sql
-- CTAs statement to create a managed table (takes longer)
CREATE OR REPLACE TABLE gizmobox.silver.customers
AS
WITH cte_max AS(
  SELECT 
    customer_id, 
    MAX(created_timestamp) AS max_created_timestamp
  FROM v_customers_distinct
  GROUP BY customer_id
)
SELECT
  CAST(t1.created_timestamp AS timestamp) created_timestamp,
  t1.customer_id,
  t1.customer_name,
  CAST(t1.date_of_birth AS date) date_of_birth,
  t1.email,
  CAST(t1.member_since AS date) member_since,
  t1.telephone
FROM v_customers_distinct t1
JOIN cte_max t2
ON t1.customer_id = t2.customer_id AND t1.created_timestamp = t2.max_created_timestamp;

In [0]:
%sql
SELECT * FROM gizmobox.silver.customers;

In [0]:
%sql
DESCRIBE EXTENDED gizmobox.silver.customers;